In [7]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [8]:
# Load data using the provided load_data.py script
def load_data():
    data0 = np.load('../DL-Project/data0.npy')
    data1 = np.load('../DL-Project/data1.npy')
    data2 = np.load('../DL-Project/data2.npy')
    lab0 = np.load('../DL-Project/lab0.npy')
    lab1 = np.load('../DL-Project/lab1.npy')
    lab2 = np.load('../DL-Project/lab2.npy')

    # Combine data and labels
    data = np.concatenate([data0, data1, data2], axis=0)
    labels = np.concatenate([lab0, lab1, lab2], axis=0)

    return data, labels

In [9]:
# Preprocess data
def preprocess_data(data, labels):
    # Normalize images to the range [0, 1]
    data = data / 255.0

    # Add channel dimension for grayscale images
    data = np.expand_dims(data, axis=-1)

    # Split data into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)

    return X_train, X_val, y_train, y_val

# Visualize data with rounded predictions
def visualize_predictions(data, labels, predictions, num_samples=5):
    plt.figure(figsize=(10, 5))
    for i in range(num_samples):
        plt.subplot(1, num_samples, i + 1)
        plt.imshow(data[i].squeeze(), cmap='gray')
        plt.title(f"Actual: {labels[i]}\nPredicted: {predictions[i]}")
        plt.axis('off')
    plt.tight_layout()
    plt.show()

In [10]:
# Define the CNN model
def build_model(input_shape):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Dropout(0.2),

        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Dropout(0.3),

        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.4),
        Dense(1, activation='linear')  
    ])

    return model

In [11]:
# Calculate accuracy with predictions
def calculate_accuracy(labels, predictions, threshold=1.0):
    correct = np.sum(np.abs(labels - predictions) <= threshold)
    accuracy = correct / len(labels)
    return accuracy


In [12]:
# Load and preprocess data
data, labels = load_data()
X_train, X_val, y_train, y_val = preprocess_data(data, labels)

# Build the model
input_shape = X_train.shape[1:]  # Get input shape from the training data
model = build_model(input_shape)

model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error', metrics=['mae'])

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=15,
    batch_size=32
)

model.save('baseline_model.keras')

/home/poorvi/miniconda3/envs/mlns/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-01-08 20:16:12.295530: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Epoch 1/15
750/750 ━━━━━━━━━━━━━━━━━━━━ 44s 57ms/step - loss: 44.3391 - mae: 5.2395 - val_loss: 23.6262 - val_mae: 3.9036
Epoch 2/15
750/750 ━━━━━━━━━━━━━━━━━━━━ 45s 60ms/step - loss: 25.0566 - mae: 4.0149 - val_loss: 16.4330 - val_mae: 3.2455
Epoch 3/15
750/750 ━━━━━━━━━━━━━━━━━━━━ 48s 64ms/step - loss: 19.8357 - mae: 3.5652 - val_loss: 15.3013 - val_mae: 3.1202
Epoch 4/15
750/750 ━━━━━━━━━━━━━━━━━━━━ 54s 72ms/step - loss: 17.4832 - mae: 3.3323 - val_loss: 17.2368 - val_mae: 3.3381
Epoch 5/15
750/750 ━━━━━━━━━━━━━━━━━━━━ 54s 72ms/step - loss: 16.1724 - mae: 3.1962 - val_loss: 12.6083 - val_mae: 2.8159
Epoch 6/15
750/750 ━━━━━━━━━━━━━━━━━━━━ 53s 71ms/step - loss: 15.8222 - mae: 3.1737 - val_loss: 16.8631 - val_mae: 3.3083
Epoch 7/15
750/750 ━━━━━━━━━━━━━━━━━━━━ 52s 70ms/step - loss: 14.4853 - mae: 3.0432 - val_loss: 13.9865 - val_mae: 2.9881
Epoch 8/15
750/750 ━━━━━━━━━━━━━━━━━━━━ 57s 76ms/step - loss: 13.8387 - mae: 2.9539 - val_loss: 10.7704 - val_mae: 2.6053
Epoch 9/15
750/750 ━━━━━

In [13]:
val_loss, val_mae = model.evaluate(X_val, y_val)
print(f"Validation Loss: {val_loss}, Validation MAE: {val_mae}")

188/188 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 11.0743 - mae: 2.6445
Validation Loss: 11.38208293914795, Validation MAE: 2.67899227142334
